In [1]:
from model import Network
from data import load_cifar10_dataloaders, whitening_transformation
import torch
import torchvision.transforms as transforms
import torch.nn as nn
# from criterion import NonGaussianityLoss, TotalVariationLoss, MutualIndependenceLoss, ReconstructImageFromFCLoss

In [2]:
# load to model
model_config = './model_config/fc2_cocktail_party_instance_wout_bias.json'
checkpoint_path = './checkpoints/121923_fc2_cocktail_party_cifar10_pretraining_wout_bias.pth'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Network(model_config)
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])
model = model.to(device)

# get val loader
normalize_mean, normalize_std = (0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)
batch_size = 8
data_path = './data'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(normalize_mean, normalize_std),
])

_, val_dataloader = load_cifar10_dataloaders(data_path, batch_size, transform)
selected_val_batch_data, selected_val_batch_label = next(iter(val_dataloader))
selected_val_batch_data = selected_val_batch_data.to(device)
selected_val_batch_label = selected_val_batch_label.to(device)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# receiving gradients
model.zero_grad()
criterion = nn.CrossEntropyLoss()
output = model(selected_val_batch_data.reshape(batch_size, -1))
loss = criterion(output, selected_val_batch_label)
loss.backward()
gradient_of_layers = []
for param in model.parameters():
    gradient_of_layers.append(param.grad.data.clone().to('cpu'))
print([x.size() for x in gradient_of_layers])

[torch.Size([256, 3072]), torch.Size([10, 256])]


In [5]:
pca_whitened_gradients = []
zca_whitened_gradients = []
for gradient in gradient_of_layers:
    pca_whitened_gradient, zca_whitened_gradient = whitening_transformation(gradient)
    pca_whitened_gradients.append(pca_whitened_gradient)
    zca_whitened_gradients.append(zca_whitened_gradient)

KeyboardInterrupt: 

In [5]:
# torch.dist(gradient_of_layers[0], pca_whitened_gradients[0])
pca_whitened_gradient[0]

tensor([ 1.8612e-01,  2.4517e-01,  5.3169e-01, -1.6703e-01, -4.8509e-01,
        -1.3563e+00,  2.1249e+00,  5.8009e-01,  2.9512e-04,  2.9512e-04,
        -1.1528e-03,  2.9620e-04,  2.9620e-04,         nan,         nan,
                nan,  9.9798e-04,  9.9798e-04, -4.4859e-04, -4.4859e-04,
         1.9309e-05,  1.9309e-05,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,  4.5593e-04,
         4.5593e-04,  1.0572e-03, -2.7668e-04, -2.7668e-04,         nan,
                nan,         nan,         nan,         nan,         nan,
        -4.9821e-04,  8.2006e-05,  8.2006e-05,  7.2980e-04,  1.7223e-04,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,  8.3325e-05,  8.3325e-05, -5.7583e-04,
        -5.7583e-04,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan, -1.3024e-04, -1.3